In [1]:
import torch #main torch module
import torch.nn as nn #neural net module
import torch.optim as optim #optimizers
import torch.nn.functional as F #functions like ReLu Sig Tanh etc
from torch.utils.data import DataLoader #help us with datasets

import torchvision.datasets as datasets #using to access std data
import torchvision.transforms as transforms #transformations
from tqdm import tqdm #progress bar

### Network Definitions

Below we will define the CNN Model

In [2]:
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels=8, 
                               kernel_size=(3,3), stride = (1,1), padding= (1,1))
        self.pool = nn.MaxPool2d(kernel_size = (2,2), stride = (2,2))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels=16 , 
                               kernel_size=(3,3), stride = (1,1), padding= (1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)
    
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        
        return x

In [3]:
model = CNN(1,10)
x = torch.randn(64, 1, 28, 28)
print(model(x).shape)

torch.Size([64, 10])


### Model Training & Eval.

In [4]:
# Set Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
device

device(type='cuda')

In [6]:
# Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10
load_model = False

In [7]:
# Loading Test Data
test_dataset = datasets.MNIST(root='D:/Datasets/',
                               train=False, 
                               transform = transforms.ToTensor(),
                               download = True)

test_loader = DataLoader(dataset = test_dataset,
                         batch_size = batch_size,
                         shuffle = True)

# Loading Data
train_dataset = datasets.MNIST(root='D:/Datasets/',
                               train=True, 
                               transform = transforms.ToTensor(),
                               download = True)

train_loader = DataLoader(dataset = train_dataset,
                         batch_size = batch_size,
                         shuffle = True)

In [8]:
#init the network
model = CNN(in_channels = 1,num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                      lr = learning_rate)

In [9]:
def epoch_accuracy_CNN(loader,model):
    #either train or test accuracy
    num_correct = 0
    num_samples = 0
    model.eval() #set to evaluation mode
    with torch.no_grad():
        #only have to check accuracy, dont compute grads
        for x,y in loader:
            x = x.to(device)
            y = y.to(device)
            #x = x.reshape(x.shape[0],-1)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    accuracy = float(num_correct)/float(num_samples)
    model.train()
    return accuracy 

In [10]:
%%time
# training loop
for epoch in range(num_epochs):
    #start of progress bar loop
    loop = tqdm(enumerate(train_loader), leave = False, total = len(train_loader))
    #tqdm leave param makes in one progress bar
    
    for batch_idx, (data, targets) in loop:       
        # data to devices
        data = data.to(device)
        targets = targets.to(device)
        
        #fwd
        scores = model(data)
        loss = criterion(scores, targets)
        
        #back
        optimizer.zero_grad() #so that it does not store prev backprop calc
        loss.backward()
    
        #gradient desc
        optimizer.step()
        
        
        
        #update progress bar
        loop.set_description(f"Epoch [{epoch}/{num_epochs}]")
        loop.set_postfix(loss = loss.item())

CPU times: user 1min 4s, sys: 2.03 s, total: 1min 6s
Wall time: 1min 2s


### Training Characteristics

In [11]:
#!pip install seaborn

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
plt.figure(figsize=(10,5));
sns.lineplot(x = range(num_epochs), y = train_accs, label='Training Accuracy');
sns.lineplot(x = range(num_epochs), y = test_accs, label='Testing Accuracy');

NameError: name 'train_accs' is not defined

<Figure size 720x360 with 0 Axes>